In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import time
import pandas as pd
import os
import requests
from collections import Counter, defaultdict
from scipy.stats import spearmanr
import nltk
from nltk.corpus import reuters
import gensim.downloader as api

# DATA LOADING & PREPROCESSING
try:
    nltk.download('reuters')
    nltk.download('punkt')
except Exception as e:
    print(f"NLTK Download Error: {e}")

def load_news_dataset(category='grain', max_vocab=2000):
    raw_sents = reuters.sents(categories=category)
    clean_corpus = [[w.lower() for w in s if w.isalpha()] for s in raw_sents]
    all_words = [w for s in clean_corpus for w in s]
    counts = Counter(all_words)
    vocab = sorted(counts, key=counts.get, reverse=True)[:max_vocab]
    vocab.append('<UNK>')
    word2idx = {w: i for i, w in enumerate(vocab)}
    return clean_corpus, vocab, word2idx

def get_skipgrams(corpus, w2i, window_size):
    """Generates training pairs dynamically based on window size."""
    data = []
    for sentence in corpus:
        for i, word in enumerate(sentence):
            if word not in w2i: continue
            target = w2i[word]
            start = max(0, i - window_size)
            end = min(len(sentence), i + window_size + 1)
            for j in range(start, end):
                if i == j or sentence[j] not in w2i: continue
                data.append((target, w2i[sentence[j]]))
    return data

def get_cooc_matrix(corpus, w2i, window_size):
    """Generates co-occurrence matrix dynamically for GloVe."""
    cooc = defaultdict(float)
    for sentence in corpus:
        for i, word in enumerate(sentence):
            if word not in w2i: continue
            w_i = w2i[word]
            start = max(0, i - window_size)
            end = min(len(sentence), i + window_size + 1)
            for j in range(start, end):
                if i == j or sentence[j] not in w2i: continue
                w_j = w2i[sentence[j]]
                cooc[(w_i, w_j)] += 1.0 / abs(i - j)
    return [(i, j, c, min(1.0, (c/100)**0.75)) for (i, j), c in cooc.items()]



[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\alsto\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alsto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# MODEL ARCHITECTURES

class SkipgramSoftmax(nn.Module): # Skip-gram with Softmax model implementation
    def __init__(self, v_size, emb_dim):
        super().__init__()
        self.embeddings = nn.Embedding(v_size, emb_dim)
        self.output = nn.Linear(emb_dim, v_size)
    def forward(self, x):
        return self.output(self.embeddings(x))

class Word2VecNeg(nn.Module): # Negative Sampling model implementation
    def __init__(self, v_size, emb_dim):
        super().__init__()
        self.v_embeddings = nn.Embedding(v_size, emb_dim)
        self.u_embeddings = nn.Embedding(v_size, emb_dim)
        self.log_sigmoid = nn.LogSigmoid()
    def forward(self, center, target, negative):
        v = self.v_embeddings(center).view(center.size(0), 1, -1)
        u = self.u_embeddings(target).view(target.size(0), 1, -1)
        n = -self.u_embeddings(negative)
        pos = torch.bmm(u, v.transpose(1, 2)).view(center.size(0), -1)
        neg = torch.bmm(n, v.transpose(1, 2)).squeeze(2)
        return -torch.mean(self.log_sigmoid(pos) + torch.sum(self.log_sigmoid(neg), 1))

class GloVeModel(nn.Module): # Glove model implementation
    def __init__(self, v_size, emb_dim):
        super().__init__()
        self.v_emb, self.u_emb = nn.Embedding(v_size, emb_dim), nn.Embedding(v_size, emb_dim)
        self.v_bias, self.u_bias = nn.Embedding(v_size, 1), nn.Embedding(v_size, 1)
    def forward(self, i, j, cooc, weights):
        dot = (self.v_emb(i) * self.u_emb(j)).sum(1)
        b_i, b_j = self.v_bias(i).squeeze(), self.u_bias(j).squeeze()
        return torch.mean(weights * torch.pow(dot + b_i + b_j - torch.log(cooc), 2))



In [3]:
# --- 3. EVALUATION LOGIC (TASK 2) ---

def evaluate_spearman(model, w2i, csv_path, is_gensim=False):
    """Calculates correlation between model dot products and human metrics."""
    if not os.path.exists(csv_path): return 0.0
    df = pd.read_csv(csv_path)
    if is_gensim:
        vecs = model
    else:
        if hasattr(model, 'v_embeddings'): v = model.v_embeddings.weight.detach().numpy()
        elif hasattr(model, 'v_emb'): v = (model.v_emb.weight + model.u_emb.weight).detach().numpy()
        else: v = model.embeddings.weight.detach().numpy()
        vecs = v
    m_sims, h_scores = [], []
    for _, row in df.iterrows():
        w1, w2, h_score = str(row['Word 1']).lower(), str(row['Word 2']).lower(), float(row['Human (mean)'])
        if is_gensim:
            if w1 in vecs and w2 in vecs:
                m_sims.append(np.dot(vecs[w1], vecs[w2]))
                h_scores.append(h_score)
        elif w1 in w2i and w2 in w2i:
            v1, v2 = vecs[w2i[w1]], vecs[w2i[w2]]
            m_sims.append(np.dot(v1, v2))
            h_scores.append(h_score)
    corr, _ = spearmanr(m_sims, h_scores)
    return corr



In [4]:
# EXECUTION LOOP FOR MULTIPLE WINDOW SIZES 

WINDOW_SIZES = [2, 5, 10]
corpus, vocab, word2idx = load_news_dataset()
performance_results = []
judgment_results = {}

for window in WINDOW_SIZES:
    print(f"\n TESTING WINDOW SIZE: {window} ")
    
    models = [
        ("Skipgram Softmax", SkipgramSoftmax(len(vocab), 100)),
        ("Skipgram NEG", Word2VecNeg(len(vocab), 100)),
        ("GloVe Scratch", GloVeModel(len(vocab), 100))
    ]

    for name, model in models:
        optimizer = optim.Adam(model.parameters(), lr=0.005)
        start_time = time.time()
        
        # Dynamic Data Generation and Training with Epoch Logging
        if "Skipgram" in name:
            pairs = get_skipgrams(corpus, word2idx, window)
            criterion = nn.CrossEntropyLoss() if "Softmax" in name else None
            for epoch in range(20): 
                total_loss = 0
                for i in range(0, len(pairs), 128):
                    batch = pairs[i:i+128]
                    if len(batch) < 128: continue
                    c, t = torch.LongTensor([p[0] for p in batch]), torch.LongTensor([p[1] for p in batch])
                    optimizer.zero_grad()
                    loss = model(c, t, torch.LongTensor(np.random.randint(0, len(vocab), (128, 5)))) if "NEG" in name else criterion(model(c), t)
                    loss.backward(); optimizer.step(); total_loss += loss.item()
                avg_loss = total_loss / (len(pairs)//128)
                print(f"  {name} (W={window}) Epoch {epoch+1}: Loss {avg_loss:.4f}")
        else: # GloVe
            cooc = get_cooc_matrix(corpus, word2idx, window)
            for epoch in range(20):
                total_loss = 0
                for i in range(0, len(cooc), 128):
                    batch = cooc[i:i+128]
                    if len(batch) < 128: continue
                    idx_i, idx_j = torch.LongTensor([p[0] for p in batch]), torch.LongTensor([p[1] for p in batch])
                    cnt, wgt = torch.FloatTensor([p[2] for p in batch]), torch.FloatTensor([p[3] for p in batch])
                    optimizer.zero_grad(); loss = model(idx_i, idx_j, cnt, wgt); loss.backward(); optimizer.step(); total_loss += loss.item()
                avg_loss = total_loss / (len(cooc)//128)
                print(f"  {name} (W={window}) Epoch {epoch+1}: Loss {avg_loss:.4f}")

        duration = time.time() - start_time
        corr = evaluate_spearman(model, word2idx, 'combined.csv')
        
        performance_results.append({
            "Model": name, "Window": window, 
            "Training Time": f"{duration:.2f}s", "Spearman Rho": f"{corr:.4f}"
        })
        
        # Keep best Spearman score per model type for the Assessment table
        if name not in judgment_results or corr > judgment_results[name]:
            judgment_results[name] = corr

# Pre-trained Gensim Benchmark
print("\nEvaluating GloVe (Gensim)")
g_g = api.load("glove-wiki-gigaword-100")
g_corr = evaluate_spearman(g_g, None, 'combined.csv', is_gensim=True)
performance_results.append({"Model": "GloVe (gensim)", "Window": "N/A", "Training Time": "N/A", "Spearman Rho": f"{g_corr:.4f}"})
judgment_results["GloVe (gensim)"] = g_corr

# DISPLAY RESULTS
print("\n PERFORMANCE SUMMARY TABLE ")
print(pd.DataFrame(performance_results).to_string(index=False))

swapped_data = [[
    judgment_results.get("Skipgram Softmax", 0),
    judgment_results.get("Skipgram NEG", 0),
    judgment_results.get("GloVe Scratch", 0),
    judgment_results.get("GloVe (gensim)", 0),
    1.0 # Y_true benchmark
]]
print("\n TABLE 1: ASSESSMENT OF HUMAN JUDGMENT CORRELATION")
print(pd.DataFrame(swapped_data, columns=["Skipgram", "NEG", "GloVe", "GloVe (gensim)", "Y_true"], index=["Spearman Rho"]))


 TESTING WINDOW SIZE: 2 
  Skipgram Softmax (W=2) Epoch 1: Loss 6.0923
  Skipgram Softmax (W=2) Epoch 2: Loss 5.6231
  Skipgram Softmax (W=2) Epoch 3: Loss 5.4678
  Skipgram Softmax (W=2) Epoch 4: Loss 5.3722
  Skipgram Softmax (W=2) Epoch 5: Loss 5.3086
  Skipgram Softmax (W=2) Epoch 6: Loss 5.2608
  Skipgram Softmax (W=2) Epoch 7: Loss 5.2263
  Skipgram Softmax (W=2) Epoch 8: Loss 5.1998
  Skipgram Softmax (W=2) Epoch 9: Loss 5.1798
  Skipgram Softmax (W=2) Epoch 10: Loss 5.1627
  Skipgram Softmax (W=2) Epoch 11: Loss 5.1493
  Skipgram Softmax (W=2) Epoch 12: Loss 5.1385
  Skipgram Softmax (W=2) Epoch 13: Loss 5.1298
  Skipgram Softmax (W=2) Epoch 14: Loss 5.1222
  Skipgram Softmax (W=2) Epoch 15: Loss 5.1162
  Skipgram Softmax (W=2) Epoch 16: Loss 5.1107
  Skipgram Softmax (W=2) Epoch 17: Loss 5.1063
  Skipgram Softmax (W=2) Epoch 18: Loss 5.1018
  Skipgram Softmax (W=2) Epoch 19: Loss 5.0987
  Skipgram Softmax (W=2) Epoch 20: Loss 5.0967
  Skipgram NEG (W=2) Epoch 1: Loss 9.9086
 